In [230]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
from sklearn.cross_validation import ShuffleSplit
%matplotlib inline

# Load the Boston housing dataset
data = pd.read_csv('train.csv')
test_data = pd.read_csv("test.csv")
prices = data['SalePrice']
features = data.drop('SalePrice', axis = 1)
print "Ames housing dataset has {} data points with {} variables each.".format(*features.shape)

Ames housing dataset has 1460 data points with 80 variables each.


In [165]:
# TODO: Minimum price of the data
minimum_price = np.amin(prices)

# TODO: Maximum price of the data
maximum_price = np.amax(prices)

# TODO: Mean price of the data
mean_price = np.mean(prices)

# TODO: Median price of the data
median_price = np.median(prices)

# TODO: Standard deviation of prices of the data
std_price = np.std(prices)

# Show the calculated statistics
print "Statistics for Ames housing dataset:\n"
print "Minimum price: ${:,.2f}".format(minimum_price)
print "Maximum price: ${:,.2f}".format(maximum_price)
print "Mean price: ${:,.2f}".format(mean_price)
print "Median price ${:,.2f}".format(median_price)
print "Standard deviation of prices: ${:,.2f}".format(std_price)

Statistics for Ames housing dataset:

Minimum price: $34,900.00
Maximum price: $755,000.00
Mean price: $180,921.20
Median price $163,000.00
Standard deviation of prices: $79,415.29


In [208]:
#check NAs
NAs = pd.concat([features.isnull().sum(), test_data.isnull().sum()], axis=1, keys=['Train', 'Test'])
NAs[NAs.sum(axis=1) > 10]

,Train,Test
LotFrontage,259,227
Alley,1369,1352
MasVnrType,8,16
MasVnrArea,8,15
BsmtQual,37,44
BsmtCond,37,45
BsmtExposure,38,44
BsmtFinType1,37,42
BsmtFinType2,38,42
FireplaceQu,690,730


In [231]:
#remove features with too many NAs
features = pd.concat([features, test_data],keys=['train', 'test'])
features.drop(['LotFrontage', 'Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu','GarageType','GarageYrBlt','GarageFinish','GarageQual', 'PoolQC', 'Fence', 'MiscFeature'], axis=1, inplace=True)
features.drop(['RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','MasVnrArea','BsmtFinSF1','BsmtFinSF2','WoodDeckSF','3SsnPorch','ScreenPorch','BsmtFullBath','BsmtHalfBath','Functional','GarageCond','LowQualFinSF','MiscVal','GarageCars'],axis=1,inplace=True)
print features.shape
NAs = features.isnull().sum()

#fill NA
features['MSSubClass'] = features['MSSubClass'].astype(str)
features['MSZoning'] = features['MSZoning'].fillna(features['MSZoning'].mode()[0])
features['Utilities'] = features['Utilities'].fillna(features['Utilities'].mode()[0])
features['BsmtUnfSF'] = features['BsmtUnfSF'].fillna(features['BsmtUnfSF'].mean())
features['TotalBsmtSF'] = features['TotalBsmtSF'].fillna(features['TotalBsmtSF'].mean())
features['Electrical'] = features['Electrical'].fillna(features['Electrical'].mode()[0])
features['KitchenQual'] = features['KitchenQual'].fillna(features['KitchenQual'].mode()[0])
features['GarageArea'] = features['GarageArea'].fillna(features['GarageArea'].mean())
features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])

(2919, 47)


In [232]:
numeric_features = features.loc[:,['LotArea','TotalBsmtSF','1stFlrSF','2ndFlrSF','GrLivArea','OpenPorchSF','EnclosedPorch','GarageArea']]
numeric_features_standardized = (numeric_features - numeric_features.mean())/numeric_features.std()

In [233]:
#get dummies
for col in features.dtypes[features.dtypes == 'object'].index:
    for_dummy = features.pop(col)
    features = pd.concat([features, pd.get_dummies(for_dummy, prefix=col)], axis=1)
#copy standardized feature
features_std = features.copy()
features_std.update(numeric_features_standardized)

In [234]:
from sklearn.cross_validation import train_test_split
from sklearn.utils import shuffle

train_features = features.loc['train'].drop('Id', axis=1).select_dtypes(include=[np.number]).values
test_features = features.loc['test'].drop('Id', axis=1).select_dtypes(include=[np.number]).values

### Splitting standardized features
train_features_st = features_std.loc['train'].drop('Id', axis=1).select_dtypes(include=[np.number]).values
test_features_st = features_std.loc['test'].drop('Id', axis=1).select_dtypes(include=[np.number]).values

print np.sum(np.isnan(train_features_st))
print np.sum(np.isnan(test_features_st))

train_features_st, train_features, train_labels = shuffle(train_features_st, train_features, prices, random_state = 10)
x_train, x_valid, y_train, y_valid = train_test_split(train_features, train_labels, test_size=0.2, random_state=0)
x_train_st, x_valid_st, y_train_st, y_valid_st = train_test_split(train_features_st, train_labels, test_size=0.2, random_state=0)


0
0


In [138]:
#feature engineering
#from sklearn.decomposition import PCA
#pca = PCA(n_components = 4, random_state = 0)
#X_train_pca = pca.fit_transform(X_train)
#X_valid_pca = pca.fit_transform(X_valid)

In [235]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state = 10)
lr.fit(x_train_st, y_train)
prediction = lr.predict(x_valid_st)
lr.score(x_valid_st, y_valid)
#performance_metric(y_valid, prediction)

0.023972602739726026

**Answer: ** If we train the model with all the data, it will try to match all data points and this can easily lead to overfitting. We need a test set of data to validate the model to be applicable to any new data, not just the known training data set. 

In [204]:
regressor = DecisionTreeRegressor(random_state=0, max_depth = 15)
regressor.fit(x_train_st, y_train)
prediction = regressor.predict(x_valid_st)
regressor.score(y_valid, prediction)

//anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: Number of features of the model must match the input. Model n_features is 173 and input n_features is 292 

In [222]:
# Produce a matrix for client data

sub = pd.DataFrame()
sub['Id'] = test_data['Id']
print np.sum(np.isnan(test_features_st))
sub['SalePrice'] = lr.predict(test_features_st)
#sub.to_csv("dt_submission.csv",index=False)

3


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').